# Tiny flowing Tensors
Implementing a small version of TensorFlow with the help of Udacity, to better understand backpropagation and differentiable graphs.

In [69]:
# fancy functions defined by Udacity, that I am a benefactor of

def topological_sort(feed_dict):
    """
    Sort the neurons in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Neuron and the value is the respective value feed to that Neuron.

    Returns a list of sorted neurons.
    """

    input_neurons = [n for n in feed_dict.keys()]

    G = {}
    neurons = [n for n in input_neurons]
    while len(neurons) > 0:
        n = neurons.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_neurons:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            neurons.append(m)

    L = []
    S = set(input_neurons)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_neurons:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_Neuron, sorted_neurons):
    """
    Performs a forward pass through a list of sorted neurons.

    Arguments:

        `output_Neuron`: A Neuron in the graph, should be the output Neuron (have no outgoing edges).
        `sorted_neurons`: a topologically sorted list of neurons.

    Returns the output Neuron's value
    """

    for n in sorted_neurons:
        n.forward()

    return output_Neuron.value

---

# Class and Subclass definitions

In [64]:
class Neuron(object):
    def __init__(self, inbound_neurons=[]):
        #print("ibn: {}".format(inbound_nodes))
        # Inbound neurons are those that the current node receives values from
        # I'm passing a LIST of neuron OBJECTS
        self.inbound_neurons = inbound_neurons
        # Outbound neurons are those that it passes the values forward to
        self.outbound_neurons = []
        # For every inbound node add an outbound neuron to that node
        # TODO: I don't quite get it. Practice coming up...!
        for n in self.inbound_neurons:
            n.outbound_neurons.append(self)
        # the value the node will eventually calculate
        self.value = None
         
    def forward(self):
        """
        Forward Propagation.
        
        Compute the output based on `inbound_nodes`
        and store the result in self.value
        """
        # implemented in the subclasses
        raise NotImplemented

In [65]:
class Input(Neuron):
    def __init__(self):
        # no inbound neurons for an input neuron
        # therefore the instantiator doesn't need inputs
        Neuron.__init__(self)
        
    def forward(self, value=None):
        # input neurons get a value input by Mr. Human
        # either explicitly or through this forward() method:
        if value is not None:
            self.value = value

In [66]:
class Add(Neuron):
    def __init__(self, neuron1, neuron2):
        Neuron.__init__(self, [neuron1, neuron2])
    
    # remember to pass `self` into the class methods!!
    def forward(self):
        """Calculates the sum of the values of the inbound nodes."""
        value_neuron1 = self.inbound_neurons[0].value
        value_neuron2 = self.inbound_neurons[1].value
        self.value = value_neuron1 + value_neuron2

In [79]:
class Linear(Neuron):
    def __init__(self, inputs, weights, bias):
        Neuron.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """Set self.value to the value of the linear function output."""
        inputs = self.inbound_neurons[0].value
        weights = self.inbound_neurons[1].value
        bias = self.inbound_neurons[2].value
        # the bias is a one-stop thing for the Neuron
        # meaning it only gets calculated once
        self.value = bias
        #print(inputs, weights, bias)
        # from here onwards we simply add the other part of the linear function(x*w)
        # to the value holding the bias:
        for i, x in enumerate(inputs):
            # calculate remaining part of the linear function
            self.value += x * weights[i]

Interestingly my code shows consistently different results than the Udacity code playground suggests that it returns.
Even though I copied the code I developed here into there. This is strange.

# Testing for sanity
I wrote a succession of (probably overly complicated) tests to check whether I am making all this up, or if there really is something wrong with Udacity's test code.

In [130]:
# create feed_dict from Udacity response
def recreate_feed_dict(response):
    """
    Extracts the values of the feed_dict used by Udacity for testing purposes.
    
    Takes as input the string returned from the Udacity Quiz in *6. Learning and Loss*
    and returns the original feed_dict and the suggested result.
    
    :param response: feedback text from modal
    :type response: str
    :return: the feed_dict used for testing
    :rtype: dict{str : list, str : list, str : int}
    :return: the suggested result, according to the feedback text
    :rtype: int
    """
    import re
    all_nums = re.findall(r"[\d]+", response)
    all_nums = [int(n) for n in all_nums]
    feed_dict = {}
    feed_dict["inputs"] = all_nums[:3]
    feed_dict["weights"] = all_nums[3:6]
    feed_dict["bias"] = all_nums[6]
    suggested_result = all_nums[-1]
    return feed_dict, suggested_result

In [137]:
def use_formula(feed_dict):
    """Calculates the weighted sums of inputs and weights, and ultimately the linear function by adding the bias.
    
    Code the formula without a detour across Neurons, to make sure the error isn't there.
    
    value = weighted_sum(x * w) + b
    
    :param feed_dict: contains values for inputs, weights, and bias
    :type feed_dict: dict{str : list, str : list, str : int}
    :return: calculated value of the linear function using the weighted sum of inputs and weights
    :rtype: int
    """
    inputs = feed_dict["inputs"]
    weights = feed_dict["weights"]
    bias = feed_dict["bias"]
    value = 0
    for x, w in zip(inputs, weights):
        value += x * w
    value += bias
    return value

def use_NN(input_feed_dict):
    """Runs a test according to Udacity's `nn.py`, using the Neuron Class I defined further up."""
    inputs, weights, bias = Input(), Input(), Input()
    f = Linear(inputs, weights, bias)
    # transforms the input_feed_dict (extracted from the error msg) into a feed_dict
    # that consists of Neuron objects.
    feed_dict = {
        inputs: input_feed_dict["inputs"],
        weights: input_feed_dict["weights"],
        bias: input_feed_dict["bias"]
    }
    graph = topological_sort(feed_dict)
    output = forward_pass(f, graph)
    return output

def compare(value1, value2):
    """Checks whether two values are the same."""
    assert (value1 == value2), "Problems brewing! 💥 "
    print("Life is good, the ☀️ is shining! Here you go, have an 🍏 !")
    
def test_reality(feedback_msg):
    """Checks whether the NN code returns the same results as the simple function calculation."""
    feed_dict, udacity_result = recreate_feed_dict(feedback_msg)
    nn_output = use_NN(feed_dict)
    formula_output = use_formula(feed_dict)
    compare(nn_output, formula_output)

In [138]:
def test_udacity_tests(feedback_msg):
    """Checks whether the output of my NN code is indeed the same as Udacity's error message suggests."""
    feed_dict, ud_result = recreate_feed_dict(feedback_msg)
    my_result = use_NN(feed_dict)
    compare(my_result, ud_result)

So I submitted a few times to collect the output messages. Interestingly enough, one time my code passed, so there must be an edge case that allows it to pass, while generally it fails.

In [139]:
feedback_list = [
    "Not quite. When given x: 2, y: 5, z: 1, weight_x: 4, weight_y: 9, weight_z: 3, bias: 7, your linear neuron outputs 64",
    "Not quite. When given x: 1, y: 8, z: 8, weight_x: 4, weight_y: 5, weight_z: 9, bias: 7, your linear neuron outputs 83",
    "Not quite. When given x: 5, y: 9, z: 5, weight_x: 0, weight_y: 1, weight_z: 4, bias: 3, your linear neuron outputs 12",
    "Not quite. When given x: 0, y: 4, z: 9, weight_x: 4, weight_y: 8, weight_z: 5, bias: 4, your linear neuron outputs 72",
    "Not quite. When given x: 6, y: 3, z: 4, weight_x: 4, weight_y: 0, weight_z: 0, bias: 2, your linear neuron outputs 42"
]

In [140]:
for msg in feedback_list:
    test_reality(msg)

Life is good, the ☀️ is shining! Here you go, have an 🍏 !
Life is good, the ☀️ is shining! Here you go, have an 🍏 !
Life is good, the ☀️ is shining! Here you go, have an 🍏 !
Life is good, the ☀️ is shining! Here you go, have an 🍏 !
Life is good, the ☀️ is shining! Here you go, have an 🍏 !


In [141]:
for msg in feedback_list:
    test_suite(msg)

AssertionError: Problems brewing! 💥 

---

I think there's a problem with the test code. Please correct me if I'm wrong.